In [1]:
import random

def hex_code_color():
    a = hex(random.randrange(0,256))
    b = hex(random.randrange(0,256))
    c = hex(random.randrange(0,256))
    a = a[2:]
    b = b[2:]
    c = c[2:]
    if len(a)<2:
        a = "0" + a
    if len(b)<2:
        b = "0" + b
    if len(c)<2:
        c = "0" + c
    z = a + b + c
    return "#" + z.upper()

In [9]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import DatetimeTickFormatter
import numpy as np
output_notebook()

# plotter
def SMA_plotter(code, df, x_axis_label):
    # create a new plot
    p = figure(title=code + " Simple Moving Averages", x_axis_label=x_axis_label, y_axis_label='Closing Price', width=800, x_axis_type='datetime')

    x_index = np.where(df.columns.values==x_axis_label)
    columns = np.delete(df.columns.values, x_index)
    print columns
    for item in columns:
        color = hex_code_color()
        if item == 'Close':
            p.line(df[x_axis_label], df[item], legend=item, line_color=color, line_width=3)
        else:
            p.line(df[x_axis_label], df[item], legend=item, line_color=color)
    
    p.xaxis.formatter=DatetimeTickFormatter(formats=dict(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    ))

    show(p, notebook_handle=True)

Loading BokehJS ...

In [55]:
# %load Moving_Average.py
"""
Created on Tue Feb 07 23:30:28 2017

@author: rkprajap
"""
# import os, datetime, logging
# import pandas as pd
# import glob
# import numpy as np

# cur_dir = os.getcwd()
# today = datetime.date.today()
# FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
# log_file = 'log\\' + str(today) + '.log'
# logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

# data_files = glob.glob("data\\*.xlsx")
# sma_list = [5, 10, 20, 50]
# scored_df = pd.DataFrame(columns=['Code']).set_index(['Code']) #this df contains final scoring


# calculate SMA scores from a given SMA list
def compute_SMA_scores(smalist, code, df):
    for sma in sma_list:
        df[str(sma)] = df['Close'].rolling(window=sma,center=False).mean()

    final_sma_values = df[[str(item) for item in smalist]].tail(1).values[0]
    sorted_sma = np.sort(sma_list)
    
    score = 0
    for index, sma in enumerate(sorted_sma):
        if df['Close'].values[0] < final_sma_values[index]:
            score = score + (index + 1)*sma
    temp_df = pd.DataFrame([[code, score]], columns=['Code', 'SMA_Score']).set_index(['Code'])
    return temp_df

#         df = stock_history[sma_columns].tail(np.max(sma_list))
#         print code, score, df['Close'].values[0] < df['SMA5'].values[0], df['Close'].values[0] < df['SMA10'].values[0], df['Close'].values[0] < df['SMA20'].values[0], df['Close'].values[0] < df['SMA50'].values[0]
#         SMA_plotter(code, df, 'Date')

In [63]:
def compute_yearly_high_low_scores(df):
    df['52WH_Score'] = 0.0
    df['52WL_Score'] = 0.0
    df = df.tail(365)

    highest_value = stock_history['Close'].max()
    lowest_value = stock_history['Close'].min()
    current_value = stock_history['Close'].tail(1).values[0]
        
    gap_2_highest = round(((highest_value - current_value)/current_value)*100,2)
    gap_2_lowest = round(((current_value - lowest_value)/current_value)*100, 2)
        
    return gap_2_highest, gap_2_lowest

In [73]:
def compute_stock_scores(code, df):
#     SMA Scores
    sma_list = [5, 10, 20, 50]
    SMA_score = compute_SMA_scores(sma_list, code, df)
    
# 365 days high and low difference score
    yearly_hl_score = compute_yearly_high_low_scores(df)
    SMA_score.set_value(code, '365H_Score', yearly_hl_score[0])
    SMA_score.set_value(code, '365L_Score', yearly_hl_score[1])
    return SMA_score
    

In [77]:
import os, datetime, logging
import pandas as pd
import glob
import numpy as np

cur_dir = os.getcwd()
today = datetime.date.today()
FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
log_file = 'log\\' + str(today) + '.log'
logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

data_files = glob.glob("data\\*.xlsx")
sma_list = [5, 10, 20, 50]
scored_df = pd.DataFrame(columns=['Code']).set_index(['Code']) #this df contains final scoring

# Start computing stats for each stock
for index, f in enumerate(data_files):
    code = f.replace('data\\', '').replace('.xlsx', '')
    stock_history = pd.read_excel(f, sheetname='Daily Data', header=0)
    stock_df = compute_stock_scores(code, stock_history)
    scored_df = scored_df.append(stock_df)
    
# normalize the scores
for column in scored_df:
    scored_df[column] = (scored_df[column]/np.max(scored_df[column])) * 10

# for buy decision, high SMA score desired, high 52WH score desired, low 52WL score desired
buy_order = scored_df.sort_values(by=['SMA_Score','365H_Score', '365L_Score'], ascending=[False, False, True])
print buy_order

            SMA_Score  365H_Score  365L_Score
Code                                         
IOC              10.0    0.652368    3.676062
TORNTPHARM       10.0    0.347490    2.338229
GLENMARK         10.0    0.285094    3.673415
SIEMENS          10.0    0.255570    2.650523
ASIANPAINT       10.0    0.222596    3.967183
DABUR            10.0    0.202001    1.472807
EMAMILTD         10.0    0.192870    2.798730
PEL              10.0    0.174912    7.113934
ASHOKLEY         10.0    0.170245    5.273257
HEROMOTOCO       10.0    0.151475    3.867937
SHREECEM         10.0    0.134025    5.298399
ZEEL             10.0    0.131894    5.455869
PIDILITIND       10.0    0.122763    2.854307
BAJAJ_AUTO       10.0    0.111907    3.963213
LICHSGFIN        10.0    0.110994    3.901019
ULTRACEMCO       10.0    0.108457    3.957920
EICHERMOT        10.0    0.098312    5.328834
BRITANNIA        10.0    0.096283    5.528649
RELIANCE         10.0    0.084717    2.766971
GODREJCP         10.0    0.079745 